In [8]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
# - OPENAI_API_KEY = ""
# - LANGCHAIN_TRACING_V2 = "true"
# - LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
# - LANGCHAIN_API_KEY = ""
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study" # Langsmith project 명

In [40]:
# 비교대상 모델ID 상수 정의
IS_HOME = False

if IS_HOME:
    AYA = "aya:8b-23-q8_0"
    LLAMA3 = "llama3:8b-instruct-q8_0"
    GEMMA = "gemma:7b-instruct-q8_0"
    PHI3 = "phi3:instruct"
else:
    AYA = "aya"
    LLAMA3 = "llama3:instruct"
    GEMMA = "gemma"
    PHI3 = "phi3:instruct"

GPT = "gpt-4o"

In [42]:
from langchain_community.llms import Ollama

llm_aya = Ollama(model=AYA, temperature=0)
llm_llama3 = Ollama(model=LLAMA3, temperature=0)
llm_gemma = Ollama(model=GEMMA, temperature=0)
llm_phi3 = Ollama(model=PHI3, temperature=0)


In [12]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(model=GPT, temperature=0)

In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    System : 너는 다음 Instruction을 잘 수행하는 assistant 이다.
    Instruction : {instruction}
    """
)

In [14]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [43]:
chain_aya = prompt | llm_aya | output_parser
chain_llama3 = prompt | llm_llama3 | output_parser
chain_gemma = prompt | llm_gemma | output_parser
chain_phi3 = prompt | llm_phi3 | output_parser
chain_gpt = prompt | llm_gpt | output_parser

param = {
    AYA : chain_aya ,
    LLAMA3 : chain_llama3 ,
    GEMMA : chain_gemma ,
    PHI3 : chain_phi3 ,
    GPT : chain_gpt ,
}

from langchain_core.runnables import RunnableParallel

chain_llms = RunnableParallel(**param)

#### (TEST) 평가대상 LLM 문제 출제 및 답안 작성

In [44]:
# chain_llms 실행 Test
responses = chain_llms.invoke({"instruction":"식혜라는 음료수에 대해 알려주세요."})

from pprint import pprint
pprint(responses)

{'aya': '식혜는 한국의 전통 음료로, 쌀로 만든 시럽이나 꿀로 단맛을 내고 때때로 마늘이나 생강과 같은 향신료를 넣어 만듭니다. '
        '그것은 종종 차가운 상태로 제공되며, 달콤하고 약간 매콤한 맛이 납니다. 식혜는 한국 문화에서 인기 있는 음료이며, 특히 '
        '여름에 인기가 많습니다. 그것은 종종 건강 증진과 소화 개선에 도움이 되는 것으로 여겨집니다.',
 'gemma': '식혜는 한국에서 유명한 음료수입니다. 주로 과일, 특히 사과를 주원으로 하고, 설탕과 향이 들어간 음료입니다. 식혜는 '
          '건강에 도움이 되고, 특히 여름에 마셔면 신선감을 느끼고 휴식을 취할 수 있습니다.',
 'gpt-4o': '식혜는 한국의 전통 음료수로, 주로 쌀과 엿기름을 사용하여 만듭니다. 식혜는 달콤하고 시원한 맛이 특징이며, 특히 '
           '명절이나 특별한 행사에서 자주 즐겨 마십니다. 다음은 식혜에 대한 주요 정보입니다:\n'
           '\n'
           '1. **재료**:\n'
           '   - **쌀**: 주로 찹쌀을 사용하지만, 멥쌀을 사용하기도 합니다.\n'
           '   - **엿기름**: 엿기름 가루를 물에 우려내어 사용합니다.\n'
           '   - **설탕**: 단맛을 더하기 위해 사용합니다.\n'
           '   - **물**: 기본적인 재료로 사용됩니다.\n'
           '\n'
           '2. **만드는 방법**:\n'
           '   1. 엿기름 가루를 물에 넣고 잘 섞은 후, 체에 걸러 엿기름 물을 만듭니다.\n'
           '   2. 찹쌀을 씻어 물에 불린 후, 밥을 짓습니다.\n'
           '   3. 지은 밥을 엿기름 물에 넣고 일정 시간 동안 발효시킵니다. 이 과정에서 밥알이 떠오르면 발효가 잘 된 '
           '것입니다.\n'
         

In [36]:
#instruction = "1990년대 한국 대중가요에 대해 알려주세요."
instruction = "미국 소련 냉전시대 때 쿠바위기에 대해 설명해주세요."

#### CASE # 1. StrOutputParser

In [45]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을 
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

    한국어로 답변해줘.

    Instruction : {instruction}
    Resonses : {responses}
    """,
    input_variables=["instruction", "responses"],
)

In [38]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt
    | llm_gpt
    | output_parser
)

- 실행(invoke)

In [39]:
response = chain_combinded.invoke({"instruction":instruction})
print(response)

### 평가 기준
1. **정확성(Accuracy)**: 제공된 정보가 사실에 근거하고 있는지 여부.
2. **관련성(Relevance)**: 질문에 대한 답변이 얼마나 관련성이 있는지.
3. **유창성(Fluency)**: 문장이 얼마나 자연스럽고 읽기 쉬운지.
4. **완전성(Completeness)**: 답변이 얼마나 포괄적이고 완전한지.

### 평가

#### aya
- **정확성**: 3점
  - 일부 곡과 가수 정보가 부정확함. 예를 들어, "네버엔딩 스토리"는 유재하의 곡이 아니며, "정글의 법칙"은 두 번째 달의 곡이 아님.
- **관련성**: 4점
  - 1990년대 한국 대중가요에 대한 다양한 정보를 제공함.
- **유창성**: 4점
  - 문장이 자연스럽고 읽기 쉬움.
- **완전성**: 4점
  - 다양한 장르와 가수를 언급하며 포괄적인 정보를 제공함.

**총점: 15점**

#### llama3:instruct
- **정확성**: 4점
  - 대부분의 정보가 정확하지만, 일부 세부사항이 부족함.
- **관련성**: 4점
  - 1990년대 K-pop에 대한 전반적인 정보를 제공함.
- **유창성**: 5점
  - 문장이 매우 자연스럽고 읽기 쉬움.
- **완전성**: 4점
  - 주요 아티스트와 그룹을 언급하며 포괄적인 정보를 제공함.

**총점: 17점**

#### gemma
- **정확성**: 3점
  - 일부 가수와 장르 정보가 부정확함. 예를 들어, "SG워드"와 "이디아"는 잘 알려지지 않은 가수임.
- **관련성**: 3점
  - 1990년대 한국 대중가요에 대한 전반적인 정보를 제공하지만, 세부사항이 부족함.
- **유창성**: 4점
  - 문장이 자연스럽고 읽기 쉬움.
- **완전성**: 3점
  - 주요 가수와 장르를 언급하지만, 정보가 다소 제한적임.

**총점: 13점**

#### gpt-4o
- **정확성**: 5점
  - 제공된 정보가 매우 정확함.
- **관련성**: 5점
  - 1990년대 한국 대중가요

#### CASE # 2. PydanticOutputParser

In [29]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class EvaluationByModel(BaseModel):
    model_id: str = Field(description="LLM 모델 이름 또는 LLM 모델 ID")
    accuracy_eval: str = Field(description="정확성(Accuracy) 평가")
    accuracy_score: int = Field(description="정확성(Accuracy) 평가 점수")
    relevance_eval: str = Field(description="관련성(Relevance) 평가")
    relevance_score: int = Field(description="관련성(Relevance) 평가 점수")
    fluency_eval: str = Field(description="유창성(Fluency) 평가")
    fluency_score: int = Field(description="유창성(Fluency) 평가 점수")
    completeness_eval: str = Field(description="완전성(Completeness) 평가")
    completeness_score: int = Field(description="관련성(Relevance) 평가 점수")

class EvaluationResponse(BaseModel):
    accuracy: str = Field(description="정확성(Accuracy) 평가 기준")
    relevance: str = Field(description="관련성(Relevance) 평가 기준")
    fluency: str = Field(description="유창성(Fluency) 평가 기준")
    completeness: str = Field(description="완전성(Completeness) 평가 기준")
    evaluation_by_model: List[EvaluationByModel] = Field(description="LLM 모델별 세부 평가 내용")
    overall: str = Field(description="종합평가")

In [30]:
# 
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
f = convert_pydantic_to_openai_function(EvaluationResponse)
f

{'name': 'EvaluationResponse',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'accuracy': {'description': '정확성(Accuracy) 평가 기준',
    'type': 'string'},
   'relevance': {'description': '관련성(Relevance) 평가 기준', 'type': 'string'},
   'fluency': {'description': '유창성(Fluency) 평가 기준', 'type': 'string'},
   'completeness': {'description': '완전성(Completeness) 평가 기준',
    'type': 'string'},
   'evaluation_by_model': {'description': 'LLM 모델별 세부 평가 내용',
    'type': 'array',
    'items': {'type': 'object',
     'properties': {'model_id': {'description': 'LLM 모델 이름 또는 LLM 모델 ID',
       'type': 'string'},
      'accuracy_eval': {'description': '정확성(Accuracy) 평가', 'type': 'string'},
      'accuracy_score': {'description': '정확성(Accuracy) 평가 점수',
       'type': 'integer'},
      'relevance_eval': {'description': '관련성(Relevance) 평가', 'type': 'string'},
      'relevance_score': {'description': '관련성(Relevance) 평가 점수',
       'type': 'integer'},
      'fluency_eval': {'description': 

In [31]:
from langchain.output_parsers import PydanticOutputParser

p_parser = PydanticOutputParser(pydantic_object=EvaluationResponse)

In [32]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을 
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

    한국어로 답변해줘.
    Format 에 맞춰서 답변해줘

    Instruction : {instruction}
    Responses : {responses}

    Format : 
    {format}
    """,
    input_variables=["instruction", "responses"],
    partial_variables={"format" : p_parser.get_format_instructions()},
)

In [33]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt 
    | llm_gpt
    | p_parser
)

- 실행(invoke)

In [34]:
response = chain_combinded.invoke({"instruction":instruction})
print(response)

accuracy='각 모델의 응답이 1990년대 한국 대중가요에 대한 사실적이고 정확한 정보를 제공하는지 평가' relevance='각 모델의 응답이 주어진 질문과 얼마나 관련성이 있는지 평가' fluency='각 모델의 응답이 문법적으로 올바르고 자연스러운지 평가' completeness='각 모델의 응답이 질문에 대해 얼마나 완전한 정보를 제공하는지 평가' evaluation_by_model=[EvaluationByModel(model_id='aya', accuracy_eval="여러 곡과 아티스트를 언급했으나, 일부 정보가 부정확함 (예: 유재하의 '네버엔딩 스토리'는 1990년대 곡이 아님)", accuracy_score=3, relevance_eval='질문에 대한 답변으로 적절하며, 1990년대 한국 대중가요에 대한 다양한 정보를 제공함', relevance_score=4, fluency_eval='문법적으로 올바르고 자연스러운 문장으로 구성됨', fluency_score=5, completeness_eval='다양한 곡과 아티스트를 언급하며, 1990년대 한국 대중가요의 특징을 잘 설명함', completeness_score=4), EvaluationByModel(model_id='llama3:instruct', accuracy_eval='1990년대 K-pop의 주요 특징과 아티스트를 정확하게 설명함', accuracy_score=5, relevance_eval='질문에 대한 답변으로 적절하며, 1990년대 K-pop의 발전과 주요 아티스트를 잘 설명함', relevance_score=5, fluency_eval='문법적으로 올바르고 자연스러운 문장으로 구성됨', fluency_score=5, completeness_eval='1990년대 K-pop의 주요 특징과 아티스트를 포괄적으로 설명함', completeness_score=5), EvaluationByModel(model_id='gemma', accuracy_eval='일부 정보가 부정확하